<h1>RNNs in TensorFlow</h1>

Das Notebook zeigt kurz wie RNNs in TensorFlow verwendet werden können.

In dem Notebook NN_RNN wurden die verschiedenen Varianten der RNN Netze gezeigt. Man hat deutlich gesehen das LSTM und GRU Zellen deutlich aufwendiger sind als herkömmliche RNN Zellen.

Dank Frameworks wie PyTorch und TensorFlow können solche Netze einfach aufgebaut und verwendet werden. Diese beinhalten die volle Implementierung dieser Zellen und bieten verschiedene Umgangsformen an.

In [1]:
# Imports. 
import numpy as np
import tensorflow as tf
import keras
from keras import layers

Übersicht der RNN Typen:

<i>Abb1</i>: Vanilla-RNN Veranschaulichung. Kontext / Memory.

<img src="../img/2_rnn.PNG" height=800 width=600>


<i>Abb2</i>: LSTM Zelle Schaubild.

<img src="../img/14_rnn.PNG" height=700 width=700>


<i>Abb3</i>: GRU Zelle Schaubild.

<img src="../img/15_rnn.PNG" height=700 width=700>

TensorFlow Keras bietet drei grundlegende Typen an: Vanilla-RNN, LSTM und GRU.: <br>
1. keras.layers.SimpleRNN 
2. keras.layers.GRU 
3. keras.layers.LSTM

Siehe auch:<br>
Enthählt weitere Details wie Optimierung.
- Recurrent layers: https://keras.io/api/layers/recurrent_layers/  [Letzter Zugriff: 29.08.2024]

Es werden Dropouts unterstützt `dropout`, `recurrent_dropout` und Loop Unrolling. Weitere Details in der Dokumentation.

<h2>In Tensorflow</h2>

In [3]:
# Erstelle Model # 

model = keras.Sequential()
# Füge Embedding hinzu. 
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Füge LSTM Zellen ein. Ein Layer mit 128 Zellen.
model.add(layers.LSTM(128))

# Output
model.add(layers.Dense(10))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Im Normalfall gibt es nur einen Outputvektor, der den Zellzustand wiedergibt. Dabei ist die Shape `(batch_size, units)`. 

Ein RNN Layer kann auch eine ganze Sequenz eines Outputs eines Samples ausgebenm dafür kann `return_sequences=True` eingestellt werden.<br>
Shape: `(batch_size, timesteps, units)`

In [5]:
# Layers haben viele Parameter
# - Siehe Dok.

model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))  # Zwei Layers

# Output 3D Tensor: (batch_size, timesteps, units)
model.add(layers.GRU(units=256, return_sequences=True))

model.add(layers.SimpleRNN(128))
model.add(layers.Dense(10))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Zusätzlich ist es möglich den internen letztendlichen Zustand/Zustände auszugeben. Das kann dann verwendet werden, um die Ausführung des RNN fortzusetze oder findet auch Anwendung in der Sequenz-to-Sequenz wie bei Übersetzungen. Dafür wird ein Encoder-Decoder, wo der Zustand des Encoders als Initialisierung des Decoders verwendet wird.

Um den Zustand auszugeben: `return_state=True` <br>
LSTM hat zwei Zustände, GRU hat einen, die ausgegeben werden können.

In [11]:
# Beispiel mit Encoder-Decoder # 
encoder_vocab = 1000
decoder_vocab = 2000

encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(encoder_input)

# Zustände in einem zusätzlichen Output #
output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(encoder_embedded)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(decoder_input)

# Jetzt diese zwei Zustände dem LSTM als Initialzustände übergeben. # 
decoder_output = layers.LSTM(64, name="decoder")(decoder_embedded, initial_state=encoder_state)

output = layers.Dense(10)(decoder_output)

model = keras.Model([encoder_input, decoder_output], output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_25CLO… │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 10)        │        650 │ keras_tensor_25C… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 650 (2.54 KB)

 Trainable params: 650 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

<h3>Cross-batch statefulness: Lange Sequenzen</h3>

Im Normalfall wird der Zustand der Zelle zurückgesetzt, wenn ein neuer Batch kommt. Jedes Batch wird als eine Sequenz betrachtet, die unabhängig von den vergangenen Sequenzen ist.

Bei sehr langen Sequenzen ist es sinnvoll diese in Teile zu schneiden. Diese Teile werden dann Stückweise in das Netz eingegeben, ohne den Zustand der Zelle zurückzusetzen. Dafür wird der Parameter `stateful=True` gesetzt.
- Sequenz: [t0, t1, ..., t400]. Daraus wird.: <br>
  s1 = [t1, t2, ..., t100] <br>
  s2 = [t101 t102, ..., t201], ...
  

Der Zustand kann auch manuell zurückgesetzt werden: `layer.reset_states()`

In [15]:
# Beispiel mit Code # 
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)
output = lstm_layer(paragraph3)

lstm_layer.reset_states()

<h3>RNN Zustände wiederverwenden</h3>

Wie erwähnt könne die Zustände eines RNNs genutzt werden, um damit neue RNNs zu initialisieren.
Mit `layer.weights()` wie sonst auch lassen sich die Zustände der Layers nicht ausgeben, dafür nutzen wir `layer.states`. Danach kann mit `new_layer(inputs, initial_state=layer.states)` der Zustand gesetzt werden.

In [16]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)

# Gebe Zustand aus # 
existing_state = lstm_layer.states

new_lstm_layer = layers.LSTM(64)
new_output = new_lstm_layer(paragraph3, initial_state=existing_state)  # Setze Zustand #

<h3>Bidirektionale RNNs</h3>

Bidirektionale RNNs sind auch wichtig, z. B. bei ENR wenn bestimmte Wörter in einem Kontext verschiedene Bedeutungen haben können (Siehe NN_RNN Notebook). 

<i>Abb4</i>: Bidirektional RNN.

<img src="../img/17_rnn.PNG" height=520 width=720>

Der Aufbau ist deutlich aufwendiger, liefert bei Texten aber auch bessere Ergebnisse. Mit Keras kann diese Architektur einfach genutzt werden.

In [18]:
model = keras.Sequential()

model.add(
    layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(5, 10))
)
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(10))

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 5, 128)         │        38,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,266 (313.54 KB)

 Trainable params: 80,266 (313.54 KB)

 Non-trainable params: 0 (0.00 B)

Weitere Details dazu findet man in der API Dokumentation.

<h1>Weitere Analysen und Details</h1>